In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Sun/ML Shock/Final project/

/content/drive/MyDrive/Sun/ML Shock/Final project


In [3]:
import sys
sys.path.insert(0,"/content/drive/MyDrive/Sun/ML Shock/Final project/")
# sys.path.insert(0,"C:/Users/Amy/Desktop/Green_Git/eegClassification/utils")

In [4]:
# from torch.utils.data import DataLoader
import pandas as pd
import os
import numpy as np

# import CustomDataset from CustomDataset.py
from CustomDataset import CustomDataset

In [5]:
# Load (train or test) data from csv file
# path = "C:/Users/Amy/Desktop/Green_Git/eegClassification/sample_data/"
path = "./data/"

data_type = "spec" #"eeg_spec"  #  "eeg_raw" #
train = True
text = "train" if train else "test"


In [6]:
# x = "train"
df = pd.read_csv(path + f"{text}.csv")# #f"/content/drive/MyDrive/Sun/ML Shock/Final project/{x}_processed.csv"

In [7]:

data_dir = (
    f"{text}_eegs/"
    if "eeg" in data_type
    else f"{text}_spectrograms/"
)
data_dir = path + data_dir

votes_cols = [
    "seizure_vote",
    "lpd_vote",
    "gpd_vote",
    "lrda_vote",
    "grda_vote",
    "other_vote",
]
label_cols = (
    ["eeg_id", "label_id", "eeg_label_offset_seconds"]
    if "eeg" in data_type
    else ["spectrogram_id", "label_id", "spectrogram_label_offset_seconds"]
)
offset = (
    ["eeg_label_offset_seconds"]
    if "eeg" in data_type
    else ["spectrogram_label_offset_seconds"]
)


In [8]:
# files = os.listdir(data_dir)
# df = df[
#     df["eeg_id" if "eeg" in data_type else "spectrogram_id"].isin(
#         [int(f.split(".")[0]) for f in files]
#     )
# ]

In [9]:
# if info_cols not in df add it and set to zero
for col in offset:
    if col not in df.columns:
        df[col] = 0
# if df does not contain "label_id" add a unique label_id
if "label_id" not in df.columns:
    df["label_id"] = range(len(df))

info = {}
df_gr = df.groupby(label_cols)
for name, group in df_gr:
    # first row of group
    info[name] = {"votes": group[votes_cols].values[0] if train else None}


In [10]:
# with open(path +f'{data_type}_{x}.pkl', 'wb') as f:
#     pickle.dump(info, f)

In [11]:
from torch.utils.data import DataLoader

In [12]:
batchsize = 16
dataset = CustomDataset(data_dir, data_type, info)
dataloader = DataLoader(dataset, batch_size=batchsize, shuffle=True)

In [13]:
count = 0
for data, label, class_probs in dataloader:
    print(data.shape, label.shape, class_probs.shape)
    count += 1
    if count > 0:
        break

torch.Size([16, 4, 299, 100]) torch.Size([16]) torch.Size([16, 6])


In [14]:
# data.shape, data.reshape(data.shape[0], data[0,:,:,0].reshape(-1,1).shape[0] , data.shape[-1] ).shape

# data[0,:,:,0].shape, data.shape[0], data[0,:,:,0].reshape(-1,1).shape[0] , data.shape[-1], 129*43


In [15]:

from scipy.special import kl_div

def get_error(y_true, y_pred):
    return np.mean(kl_div(y_true, y_pred))


In [16]:
# create a data scaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pickle

count = 0
e_count = 0
error = []

# use the partial function to create a scaler
standard_scaler = StandardScaler()
min_max_scaler = MinMaxScaler()
for data, _, y_true in dataloader:
  try:
    scale = standard_scaler.scale_
    min = min_max_scaler.data_min_
    max = min_max_scaler.data_max_
  except:
    scale = None

  standard_scaler.partial_fit(data.reshape(data.shape[0], -1 ))
  min_max_scaler.partial_fit(data.reshape(data.shape[0], -1 ))

  scale_new = standard_scaler.scale_
  min_new = min_max_scaler.data_min_
  max_new = min_max_scaler.data_max_
  if scale is not None:
    change = np.abs(scale_new - scale)
    change_min = np.abs(min_new - min)
    change_max = np.abs(max_new - max)
    print(f"Std: {np.max(change):.4e} MinMax: {np.max([np.max(change_min),np.max(change_max)]):.4e}")
  count +=1

  if count%10 == 0:
    with open(path +f'{data_type}_standard_scaler.pkl', 'wb') as f:
      pickle.dump(standard_scaler, f)
    with open(path + f'{data_type}_min_max_scaler.pkl', 'wb') as f:
        pickle.dump(min_max_scaler, f)
    print("saved", count)

  for i in range(y_true.shape[0]):
        error.append(get_error(y_true[i].numpy(), [1/6 for _ in range(6)]))
        e_count += 1

  if count > 107000/16*0.03:
    break

print(sum(error)/e_count)

Std: 2.0436e+00 MinMax: 1.2140e+01
Std: 1.6476e+00 MinMax: 1.1888e+01
Std: 1.2304e+00 MinMax: 1.1546e+01
Std: 2.0574e+00 MinMax: 1.3168e+01
Std: 4.7847e-01 MinMax: 5.6890e+00
Std: 4.2228e-01 MinMax: 7.2425e+00
Std: 6.6150e-01 MinMax: 9.3159e+00
Std: 4.9686e-01 MinMax: 8.3381e+00
Std: 4.6635e-01 MinMax: 8.9079e+00
saved 10
Std: 6.8080e-01 MinMax: 9.0579e+00
Std: 5.4627e-01 MinMax: 9.1990e+00
Std: 6.1378e-01 MinMax: 1.1225e+01
Std: 2.9535e-01 MinMax: 7.0567e+00
Std: 6.1977e-01 MinMax: 9.7428e+00
Std: 4.0825e-01 MinMax: 8.6789e+00
Std: 7.8022e-02 MinMax: 2.0180e+00
Std: 1.7701e-01 MinMax: 4.2079e+00
Std: 1.4132e-01 MinMax: 4.3757e+00
Std: 3.1198e-01 MinMax: 8.8385e+00
saved 20
Std: 7.0555e-02 MinMax: 2.1683e+00
Std: 2.3546e-01 MinMax: 7.5817e+00
Std: 2.4311e-01 MinMax: 7.7252e+00
Std: 2.1953e-01 MinMax: 8.3588e+00
Std: 5.4665e-02 MinMax: 1.2228e+00
Std: 2.1426e-01 MinMax: 6.4063e+00
Std: 1.8357e-01 MinMax: 6.1767e+00
Std: 1.2828e-01 MinMax: 3.8873e+00
Std: 2.4443e-01 MinMax: 8.9935e+00
St